<p style="font-weight:bold;"> <span style="font-size: 36px"> Reimport of Transactional Data Test</span> </p>

In [0]:
//#!eval-notebook "../Initialization/InitSystemorphBaseToMemory"

# Check Method

In [0]:
public async Task<ActivityLog> ReimportAndCompare((string transactionalData, string importFormat) importData, DataNodeState[] dataNodeStates)
{
    var workspace = Workspace.CreateNew();
    workspace.Initialize(x => x.FromSource(DataSource).DisableInitialization<DataNodeState>().DisableInitialization<IfrsVariable>().DisableInitialization<RawVariable>());
    await workspace.UpdateAsync<DataNodeState>(dataNodeStates);

    Activity.Start();
    var importActivity = await Import.FromString(importData.transactionalData).WithFormat(importData.importFormat).WithTarget(workspace).ExecuteAsync();
    if(importActivity.RepeatOnce().ToArray().HasErrors()) return importActivity;
    
    var firstImportRawVariable = workspace.Query<RawVariable>().ToArray();
    var firstImportIfrsVariable = workspace.Query<IfrsVariable>().ToArray();

    var reimportActivity = await Import.FromString(importData.transactionalData).WithFormat(importData.importFormat).WithTarget(workspace).ExecuteAsync();
    if(reimportActivity.RepeatOnce().ToArray().HasErrors()) return reimportActivity;

    var secondImportRawVariable = workspace.Query<RawVariable>().ToArray();
    var secondImportIfrsVariable = workspace.Query<IfrsVariable>().ToArray();

    List<string> errors = new List<string>();
    var missingRawVariables = firstImportRawVariable.Except(secondImportRawVariable, RawVariableComparer.Instance(ignoreValues: false));
    var missingIfrsVariables = firstImportIfrsVariable.Except(secondImportIfrsVariable, IfrsVariableComparer.Instance(ignoreValues: false));
    var extraRawVariables = secondImportRawVariable.Except(firstImportRawVariable, RawVariableComparer.Instance(ignoreValues: false));
    var extraIfrsVariables = secondImportIfrsVariable.Except(firstImportIfrsVariable, IfrsVariableComparer.Instance(ignoreValues: false));

    if(missingRawVariables.Any()) errors.Add($"Missing RawVariables after the reimport: {string.Join("\n", missingRawVariables.Select(x=>"and " + x.ToIdentityString().ToString()+x.Values.First().ToString()))}.");
    if(missingIfrsVariables.Any()) errors.Add($"Missing IfrsVariables after the reimport: {string.Join("\n", missingIfrsVariables.Select(x=>"and " + x.ToIdentityString().ToString()+x.Values.First().ToString()))}.");
    if(extraRawVariables.Any()) errors.Add($"Extra RawVariables after the reimport: {string.Join("\n", extraRawVariables.Select(x=>"and " + x.ToIdentityString().ToString()+x.Values.First().ToString()))}.");
    if(extraIfrsVariables.Any()) errors.Add($"Extra IfrsVariables after the reimport: {string.Join("\n", extraIfrsVariables.Select(x=>"and " + x.ToIdentityString().ToString()+x.Values.First().ToString()))}.");
    
    if(errors.Any()) ApplicationMessage.Log(Error.Generic, string.Join("\n", errors));

    ApplicationMessage.Log(Warning.Generic, "Count of IfrsVariables in workspace: " + secondImportIfrsVariable.Count());
    ApplicationMessage.Log(Warning.Generic, "Count of RawVariables in workspace: " + secondImportRawVariable.Count());
    workspace.Dispose();
    return Activity.Finish();
}

# Data Preparation

Create:
- Import args for current year (2021)
- Import args for previous year (2020)
- Partition by Reporting Node (using details from most recent args)

In [0]:
var args = new ImportArgs("CH", 2021, 3, Periodicity.Quarterly, null, null);
var previousArgs = new ImportArgs("CH", 2020, 12, Periodicity.Quarterly, null, null);

var partitionByReportingNode = new PartitionByReportingNode { Id = (Guid)(await DataSource.Partition.GetKeyForInstanceAsync<PartitionByReportingNode>(args)), ReportingNode = args.ReportingNode };

Create 5 data node states:
- dt11StatePrevious
- dtr11StatePrevious
- dt11State
- dtr11State
- dt13State

In [0]:
var dt11StatePrevious = new DataNodeState { DataNode = "DT1.1", State = State.Active, Year = previousArgs.Year, Month = previousArgs.Month, Partition = partitionByReportingNode.Id };
var dtr11StatePrevious = new DataNodeState { DataNode = "DTR1.1", State = State.Active, Year = previousArgs.Year, Month = previousArgs.Month, Partition = partitionByReportingNode.Id };
var dt11State = new DataNodeState { DataNode = "DT1.1", State = State.Active, Year = args.Year, Month = args.Month, Partition = partitionByReportingNode.Id };
var dtr11State = new DataNodeState { DataNode = "DTR1.1", State = State.Active, Year = args.Year, Month = args.Month, Partition = partitionByReportingNode.Id };
var dt13State = new DataNodeState { DataNode = "DT1.3", State = State.Active, Year = args.Year, Month = args.Month, Partition = partitionByReportingNode.Id };

# Test Transactional Data

## Reimport Openings for DataNode at Inception

Create openings for a data node at inception (DT1.1) and import more than once.

In [0]:
var openings = @"
@@Main,,,,
ReportingNode,Year,Month,,
CH,2021,3,,
@@Opening,,,,
DataNode,EstimateType,AmountType,AccidentYear,Value
DT1.1,C,,,72.2
DT1.1,AA,PR,,-1.5
DT1.1,OA,PR,,1.5
";

In [0]:
var activity = await ReimportAndCompare((openings, ImportFormats.Opening), new [] { dt11State, dtr11State });
activity

In [0]:
activity.Status.Should().Be(ActivityLogStatus.Succeeded);

## Reimport Actuals for Both: DataNode at Inception and DataNode Not at Inception

Create an actuals file that contains data for both a data node at inception (DT1.3) and a data node not at inception (DT1.1). 

Import this file more than once.

In [0]:
var actuals = @"
@@Main,,,,,
ReportingNode,Year,Month,,,
CH,2021,3,,,
@@Actual,,,,,
DataNode,AocType,AmountType,EstimateType,AccidentYear,Value
DT1.1,CF,PR,A,,90
DT1.1,CF,ICO,A,,-6
DT1.1,CF,NIC,A,,-70
DT1.1,CF,ACA,A,,-10
DT1.1,CF,AEA,A,,-5
DT1.3,CF,PR,A,,90
DT1.3,CF,ICO,A,,-6
DT1.3,CF,NIC,A,,-70
DT1.3,CF,ACA,A,,-10
DT1.3,CF,AEA,A,,-5";

In [0]:
var activity = await ReimportAndCompare((actuals, ImportFormats.Actual), new[] {dt11StatePrevious, dtr11StatePrevious, dt13State});
activity

In [0]:
activity.Status.Should().Be(ActivityLogStatus.Succeeded);

## Reimport Cashflows for Linked Reinsurance Not at Inception

Create a cashflows file that contains data for a data node not at inception (DTR1.1). 

Import this file more than once.

In [0]:
var cashflows = @"
@@Main,,,,,,,,,,,,,,,,,,,,,,,,,,
ReportingNode,Year,Month,,,,,,,,,,,,,,,,,,,,,,,,
CH,2021,3,,,,,,,,,,,,,,,,,,,,,,,,
@@Cashflow,,,,,,,,,,,,,,,,,,,,,,,,,,
DataNode,AmountType,EstimateType,AocType,Novelty,AccidentYear,Values0,Values1,Values2,Values3,Values4,Values5,Values6,Values7,Values8,Values9,Values10,Values11,Values12,Values13,Values14,Values15,Values16,Values17,Values18,Values19,Values20
DTR1.1,PR,BE,CL,C,,0,-57.5,0,0,-57.5,0,0,-57.5,0,0,-57.5,0,0,0,0,0,0,0,0,0,0
DTR1.1,CU,P,CL,C,,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
DTR1.1,NIC,BE,CL,C,,15,15,15,15,15,15,15,15,15,15,15,15,0,0,0,0,0,0,0,0,0
DTR1.1,,RA,CL,C,,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,0,0,0,0,0,0,0,0,0";

In [0]:
var activity = await ReimportAndCompare((cashflows, ImportFormats.Cashflow), new[] {dt11StatePrevious, dtr11StatePrevious});
activity

In [0]:
activity.Status.Should().Be(ActivityLogStatus.Succeeded);